# Setting up the environment

In [ ]:
# Import and/or install libraries

import subprocess, os

try:
    import geemap, ee
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'geemap'])
    import geemap, ee

try:
    import rio_tiler.io
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'rio-tiler'])
    import rio_tiler.io

In [ ]:
# Connect to Google Drive to access files

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Connect to Google Earth Engine if neccessary

service_account = os.environ.get('GOOGLE_SERVICE_ACCOUNT')
credentials = ee.ServiceAccountCredentials(service_account, os.environ.get('GOOGLE_PATH_TO_KEY'))
ee.Initialize(credentials)

# Getting started

We will start by using Google Dynamic World and using the built layer. It seems like we should also filter the certainty to be greater than 0.06 (highest false positive in the grassland had a certainty of 0.0569).

In [ ]:
people_6 = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate('2021-01-01', '2022-01-01').median().select('built').gt(0.6).selfMask()
people_3 = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate('2021-01-01', '2022-01-01').median().select('built').gt(0.6).selfMask()

people = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate('2021-01-01', '2022-01-01').median().select('label').eq(6).selfMask()
certainty_mask = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate('2021-01-01', '2022-01-01').median().select('built').gt(0.06).selfMask()

best_people = people.mask(certainty_mask).eq(1).selfMask()

people_c = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate('2021-01-01', '2022-01-01').median().select('built').mask(people)

# Google Open Buildings

In [ ]:
buildings = ee.FeatureCollection('GOOGLE/Research/open-buildings/v2/polygons')

t_060_065 = buildings.filter('confidence >= 0.60 && confidence < 0.65');
t_065_070 = buildings.filter('confidence >= 0.65 && confidence < 0.70');
t_gte_070 = buildings.filter('confidence >= 0.70');


# Map

In [ ]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')
# Map.addLayer(people_6, {'min':0, 'max':1, 'palette':['white','black']}, 'people_6')
# Map.addLayer(people_3, {'min':0, 'max':1, 'palette':['white','red']}, 'people_3')
# Map.addLayer(people, {'min':0, 'max':1, 'palette':['white','blue']}, 'people')
Map.addLayer(best_people, {'min':0, 'max':1, 'palette':['white','red']}, 'best_people')
Map.addLayer(t_060_065, {'color': 'FF0000'}, 'Buildings confidence [0.60; 0.65)');
Map.addLayer(t_065_070, {'color': 'FFFF00'}, 'Buildings confidence [0.65; 0.70)');
Map.addLayer(t_gte_070, {'color': '00FF00'}, 'Buildings confidence >= 0.70');

# Map.addLayer(gdw, {'min':0, 'max':8, 'palette':['419bdf', '397d49', '88b053', '7a87c6', 'e49635', 'dfc35a', 'c4281b', 'a59b8f', 'b39fe1']}, 'GDW')
# Map.addLayer(people_c, {'min':0, 'max':0.1, 'palette':['white','blue']}, 'people_c')

Map.setCenter(31.944, -25.087, 16)
Map
